# Prise en main de PySpark

## Installation et configuration

**Question 1**

Exécutez la cellule ci-dessous pour lancer l'installation de PySpark

In [ ]:
#install pyspark
!pip install pyspark

**Question 2**

Exécutez la cellule ci-dessous pour connaître le nombre de coeurs de calcul qui vous sont attribués dans Google Colab

In [ ]:
from os import cpu_count
# get the number of logical cpu cores
n_cores = cpu_count()
# report the number of logical cpu cores
print(f'Number of Logical CPU cores: {n_cores}')

**Question 3**

Ci-dessous, créer une `SparkSession` pour une application nommée "Test de PySpark" et lancée en local avec le nombre de coeurs trouvé ci-dessus.

## Quelques tests sur les RDDs

### Transformations, actions et persistence

**Question 4**

Ci-dessous, créer deux RDDs (nommés `my_rdd1` et `my_rdd2`) à partir de deux listes Python de 6 éléments chacune.

Afficher ces `RDD` avec `print()` ainsi que leur nombre de partitions.

In [ ]:
import time
from operator import add

def carre_slow(x):
  time.sleep(1)
  return x**2


**Question 5**

Executer la cellule de code ci-dessus, puis utiliser la méthode `map` des RDDs pour générer un nouveau RDD nommé `my_new_rdd1` contenant les carrés des valeurs de `my_rdd1`, en utilisant la fonction `carre_slow` définie ci-dessus.

Mesurer le temps d'exécution (`%%time`) et conclure : les traitements ont-ils réellement été exécutés à cet endroit ?

Afficher le nouveau RDD généré et son nombre de partitions.

In [ ]:
%%time


**Question 6**

En mesurant le temps d'exécution, utiliser la méthode `collect` pour récupérer et afficher le contenu du RDD `my_new_rdd1` dans le *driver program* (programme principal, ici celui du Notebook).

Conclure quant aux traitements lancés au moment de l'appel à `collect` et à la nature de cette méthode : est-ce une transformation ou une action ?

In [ ]:
%%time


**Question 7**

En mesurant les temps d'exécution, calculer à l'aide de la méthode `reduce` la somme des carrés contenus dans `my_new_rdd1` (on pourra utiliser la fonction `add` du module `operator` importée ci-dessus).

Qu'observez-vous au niveau du temps d'exécution ?

In [ ]:
%%time


**Question 8**

De la même manière, générer un nouveau RDD nommé `my_new_rdd2` contenant les carrés des valeurs de `my_rdd2`, en utilisant la fonction `carre_slow` puis appeler la méthode `persist` sur ce nouveau RDD.

Dans la seconde cellule, en mesurant le temps d'exécution, afficher le contenu de `my_new_rdd2` en utilisant la méthode `collect`.

In [ ]:
%%time


**Question 9**

En mesurant les temps d'exécution, calculer à l'aide de la méthode `reduce`la somme des carrés contenus dans `my_new_rdd2`.

Qu'observez-vous cette fois-ci au niveau du temps d'exécution ?

Conclure : à votre avis, à quoi sert la méthode `persist` appelée ci-dessus sur `my_new_rdd2` ? Vérifier en consultant la [documentation en ligne](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.persist.html#pyspark.RDD.persist).

In [ ]:
%%time


## Utilisation de Spark SQL

### Lecture et mise en forme des données

*Remarque : lors des différentes transformations de votre DataFrame il est conseillé d'afficher le résultat de vos traitements avant de remplacer la DataFrame*

**Question 10**

Charger le fichier `cereal.csv` dans une DataFrame Spark SQL nommée `sdf1` (une description des données peut être trouvée [ici](https://www.kaggle.com/datasets/crawford/80-cereals/)).

Vérifier que vous récupérer bien les en-têtes de colonne. Dans le cas contraire, adapter l'appel utilisé pour la lecture des données du fichier.

Afficher le nombre de lignes contenus dans cette DataFrame, son nombre de partitions (en passant par le RDD sous-jacent) et ses 50 premières lignes.

**Question 11**

Afficher le schéma de la DataFrame avec la méthode `printSchema` et vérifier que vous obtenez les bons types. Dans le cas contraire, adapter l'appel utilisé pour la lecture des données du fichier afin d'inférer automatiquement le schéma.

Afficher également l'attribut `schema` de la DataFrame avec `print()`. De quel type est cet objet ? Quel type d'objets contient-il ?

**Question 12**

En une seule instruction, supprimer la colonne intitulée `"shelf"` puis renommer l'ensemble des colonnes restant (en utilisant une des méthodes `toDF` ou `withColumnRenamed`) avec, dans l'ordre, les intitulés suivants :

```python
"Nom", "Fabricant","Type", "Calories","Protéines", "Graisse","Sodium","Fibres",\
"Glucides","Sucre","Potassium", "Vitamines", "Poids","Volume","Evaluation"
```

Afficher la nouvelle DataFrame obtenue.

**Question 13**

En utilisant les dictionnaires `dic_fabricant` et `dic_type` ci-dessous, remplacer à l'aide de la méthode `replace` les valeurs des colonnes `Fabricant` et `Type` par les nouvelles valeurs associées.

Afficher la nouvelle DataFrame obtenue.

In [ ]:
dic_fabricant = {"A":"American Home Food Products",
    "G" : "General Mills",
    "K" : "Kelloggs",
    "N" : "Nabisco",
    "P" : "Post",
    "Q" : "Quaker Oats",
    "R" : "Ralston Purina"}

dic_type = {"C" : "froid", "H" : "chaud"}



**Question 14**

En utilisant la méthode `withColumns`, convertir les poids donnés en *ounces* (onces) en grammes (1 once = 28,35 grammes) et les volumes donnés en *cups* (tasses) en litres (1 tasse = 0,25 litres).

### Re-partitionnement et écriture des nouvelles données

**Question 15**

A l'aide de la méthode `repartitionByRange` re-partitionner la DataFrame `sdf1` en 2 partitions en fonction du fabricant.

Afficher le nombre de partitions de la nouvelles DataFrame `sdf1` et ses 50 premières lignes. Comparer à l'affichage obtenu à la question précdente.

**Question 16**

Ecrire la nouvelles Dataframe re-partitionnée `sdf1` dans un répertoire `./cereal_french` au format `csv` avec les en-têtes. Observer le résutat obtenu.

*Remarque : Spark utilise par défaut le système de nommage de fichier de Hadoop*

D'après le contenu de chaque fichier obtenu, comment ont été faites les partitions de `sdf1` ?

### Filtrage, tri, groupements et agrégation.

**Question 17**

Afficher les lignes correspondant aux céréales avec des glucides strictement inférieurs à 10 grammes (on excluera également les valeurs négatives de glucides qui correspondent à des informations non renseignées). Ordonner les résultats suivant les glucides.

**Question 18**

Afficher le nombre de céréales par type (chaud/froid).

**Question 19**

Créer une nouvelle DataFrame `sdf2` contenant les moyennes des grandeurs numériques par fabricant et renommer les colonnes de cette DataFrame avec dans l'ordre :
```
"Fabricant","Calories","Protéines","Graisse","Sodium","Fibres",
"Glucides","Sucre","Potassium", "Vitamines", "Poids","Volume","Evaluation"
```

Afficher le nombre de partitions de cette DataFrame `sdf2` et son contenu.

**Question 20**

En utilisant la méthode `coalesce` rassembler la DataFrame `sdf2` en une seule partition puis l'écrire dans un répertoire `./fabricant_valeurs_moyennes` au format `csv` (avec les en-têtes). Observer le résutat obtenu.

**Question 21**

Exécuter la cellule de code ci-dessous pour arrêter votre `SparkSession` (on suppose que votre objet `SparkSession` s'appelle `spark`).

In [ ]:
spark.stop()